# Preprocessing v2 (External-style SMILES featurization)

This notebook implements the SMILES → features pipeline used in `predicting-melting-point-external-dataset-score.ipynb`, adapted to this repo’s paths.

**Outputs** (relative to this notebook):
- `result/data/melting_point_features.csv`
- (optional) `result/data/melting_point_features.parquet`

Notes:
- Canonicalizes SMILES and drops invalid/duplicate molecules.
- Computes RDKit 2D descriptors, fragment counts, Morgan+MACCS bits, VSA bins, charge stats, cheap interaction features, and optional cached 3D shape features.
- If a `test.csv` exists, aligns train/test feature columns like the external notebook.

In [1]:
import json
import subprocess
import sys

def ensure_package(import_name: str, install_name: str | None = None) -> None:
    """Install a pip package if missing (best-effort)."""
    install_name = install_name or import_name
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'list', '--format=json'],
        check=True,
        capture_output=True,
        text=True,
    )
    installed = {pkg['name'].lower() for pkg in json.loads(result.stdout)}
    # Some packages have different import vs distribution names (e.g., rdkit-pypi -> rdkit)
    if import_name.lower() in installed or (install_name and install_name.lower() in installed):
        print(f'{import_name} already installed.')
        return
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', install_name])
    print(f'Installed {install_name} (import as {import_name}).')

for pkg in [
    ('numpy', None),
    ('pandas', None),
    ('matplotlib', None),
    ('joblib', None),
    # rdkit wheels are commonly available as rdkit-pypi (import name is rdkit)
    ('rdkit', 'rdkit-pypi'),
]:
    ensure_package(*pkg)

numpy already installed.
pandas already installed.
matplotlib already installed.
joblib already installed.
matplotlib already installed.
joblib already installed.
rdkit already installed.
rdkit already installed.


In [2]:
from __future__ import annotations

from pathlib import Path
from typing import Any, Callable, Dict, Iterable, Optional, Tuple

import multiprocessing as mp
import numpy as np
import pandas as pd

from joblib import Parallel, delayed

from rdkit import Chem, rdBase, RDLogger
from rdkit.Chem import AllChem, Crippen, Descriptors, Fragments, Lipinski, rdMolDescriptors, rdFingerprintGenerator
from rdkit.Chem.MACCSkeys import GenMACCSKeys
from rdkit.Chem.EState import AtomTypes as EAtomTypes

try:
    from rdkit.Chem.Scaffolds import MurckoScaffold
except Exception:
    MurckoScaffold = None

RDLogger.DisableLog('rdApp.*')
rdBase.DisableLog('rdApp.*')

DATA_DIR = Path('../../../main-data')
TRAIN_PATH = DATA_DIR / 'train.csv'
TEST_PATH = DATA_DIR / 'test.csv'  # optional
OUTPUT_DIR = Path('result/data')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

MORGAN_BITS = 512
MORGAN_RADIUS = 2
USE_MACCS = True
COMPUTE_3D = True
MAX_ITERS_3D = 0  # 0 = no optimization, >0 enables a short UFF optimize

In [3]:
df_train = pd.read_csv(TRAIN_PATH)
print('train:', df_train.shape)
display(df_train.head(3))

df_test = None
if TEST_PATH.exists():
    df_test = pd.read_csv(TEST_PATH)
    print('test:', df_test.shape)
    display(df_test.head(3))
else:
    print(f'No test file at {TEST_PATH} (skipping test featurization).')

train: (2662, 427)


,id,SMILES,Tm,Group 1,Group 2,Group 3,Group 4,Group 5,Group 6,Group 7,...,Group 415,Group 416,Group 417,Group 418,Group 419,Group 420,Group 421,Group 422,Group 423,Group 424
0,2175,FC1=C(F)C(F)(F)C1(F)F,213.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1222,c1ccc2c(c1)ccc3Nc4ccccc4c23,407.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2994,CCN1C(C)=Nc2ccccc12,324.15,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


No test file at ../../../main-data/test.csv (skipping test featurization).


In [4]:
def canonicalize_smiles(smiles: Any) -> Optional[str]:
    try:
        if smiles is None or (isinstance(smiles, float) and np.isnan(smiles)):
            return None
        mol = Chem.MolFromSmiles(str(smiles))
        if mol is None:
            return None
        return Chem.MolToSmiles(mol, canonical=True)
    except Exception:
        return None

def canonicalize_frame(df: pd.DataFrame, smiles_col: str = 'SMILES') -> pd.DataFrame:
    df = df.copy()
    df[smiles_col] = df[smiles_col].apply(canonicalize_smiles)
    before = len(df)
    df = df.dropna(subset=[smiles_col]).drop_duplicates(subset=[smiles_col], keep='last').reset_index(drop=True)
    print(f'Canonicalized {before} -> {len(df)} unique molecules')
    return df

df_train = canonicalize_frame(df_train, 'SMILES')
if df_test is not None:
    df_test = canonicalize_frame(df_test, 'SMILES')

assert 'SMILES' in df_train.columns
assert 'Tm' in df_train.columns

Canonicalized 2662 -> 2660 unique molecules


In [5]:
def _safe(fn: Callable[..., Any], default: Any = 0.0) -> Callable[..., Any]:
    def wrap(*args: Any, **kwargs: Any) -> Any:
        try:
            return fn(*args, **kwargs)
        except Exception:
            return default
    return wrap

def drop_constant_and_duplicate_columns(df: pd.DataFrame) -> pd.DataFrame:
    nunique = df.nunique(dropna=False)
    constant_cols = nunique[nunique <= 1].index.tolist()
    df = df.drop(columns=constant_cols)
    df = df.loc[:, ~df.T.duplicated(keep='first')]
    return df

def reduce_memory_usage(df: pd.DataFrame) -> pd.DataFrame:
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    for col in df.columns:
        col_type = df[col].dtype
        if str(col_type) in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isna(c_min) or pd.isna(c_max):
                continue
            if str(col_type).startswith('int'):
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
            else:
                if c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
        elif col_type == object:
            num_unique = df[col].nunique(dropna=False)
            if num_unique / max(len(df), 1) < 0.5:
                df[col] = df[col].astype('category')
    return df

def _count_atoms(m: Optional[Chem.Mol], symbols: Iterable[str]) -> int:
    if m is None:
        return 0
    s = set(symbols)
    return sum(1 for a in m.GetAtoms() if a.GetSymbol() in s)

def _largest_ring_size(m: Optional[Chem.Mol]) -> int:
    if m is None:
        return 0
    ri = m.GetRingInfo()
    return max((len(r) for r in ri.AtomRings()), default=0)

def count_explicit_h(m: Optional[Chem.Mol]) -> int:
    if m is None:
        return 0
    mH = Chem.AddHs(m)
    return sum(1 for a in mH.GetAtoms() if a.GetSymbol() == 'H')

def gasteiger_stats(m: Optional[Chem.Mol]) -> Dict[str, float]:
    if m is None:
        return {'Gasteiger_q_sum': 0.0, 'Gasteiger_q_abs_sum': 0.0, 'Gasteiger_q_min': 0.0, 'Gasteiger_q_max': 0.0, 'Gasteiger_q_std': 0.0}
    mH = Chem.AddHs(m)
    try:
        AllChem.ComputeGasteigerCharges(mH)
    except Exception:
        return {'Gasteiger_q_sum': 0.0, 'Gasteiger_q_abs_sum': 0.0, 'Gasteiger_q_min': 0.0, 'Gasteiger_q_max': 0.0, 'Gasteiger_q_std': 0.0}
    vals: list[float] = []
    for a in mH.GetAtoms():
        try:
            v = float(a.GetProp('_GasteigerCharge')) if a.HasProp('_GasteigerCharge') else 0.0
        except Exception:
            v = 0.0
        if pd.isna(v) or v in (float('inf'), float('-inf')):
            v = 0.0
        vals.append(v)
    arr = np.asarray(vals, dtype=float)
    return {
        'Gasteiger_q_sum': float(arr.sum()),
        'Gasteiger_q_abs_sum': float(np.abs(arr).sum()),
        'Gasteiger_q_min': float(arr.min(initial=0.0)),
        'Gasteiger_q_max': float(arr.max(initial=0.0)),
        'Gasteiger_q_std': float(arr.std(ddof=0)),
    }

def _smiles_morphology(smi: str) -> Dict[str, int]:
    if not smi:
        return {'SMI_len': 0, 'SMI_branches': 0, 'SMI_ringDigits': 0, 'SMI_stereoAt': 0, 'SMI_ezSlashes': 0}
    return {
        'SMI_len': len(smi),
        'SMI_branches': smi.count('('),
        'SMI_ringDigits': sum(ch.isdigit() for ch in smi),
        'SMI_stereoAt': smi.count('@'),
        'SMI_ezSlashes': smi.count('/') + smi.count('\\'),
    }

def _estate_stats(m: Optional[Chem.Mol]) -> Dict[str, float]:
    if m is None:
        return {'EState_sum': 0.0, 'EState_mean': 0.0, 'EState_max': 0.0, 'EState_min': 0.0, 'EState_std': 0.0}
    try:
        vals = EAtomTypes.EStateIndices(m)
        if not vals:
            return {'EState_sum': 0.0, 'EState_mean': 0.0, 'EState_max': 0.0, 'EState_min': 0.0, 'EState_std': 0.0}
        arr = np.asarray(vals, dtype=float)
        return {
            'EState_sum': float(arr.sum()),
            'EState_mean': float(arr.mean()),
            'EState_max': float(arr.max()),
            'EState_min': float(arr.min()),
            'EState_std': float(arr.std(ddof=0)),
        }
    except Exception:
        return {'EState_sum': 0.0, 'EState_mean': 0.0, 'EState_max': 0.0, 'EState_min': 0.0, 'EState_std': 0.0}

def _bond_order(b: Chem.Bond) -> float:
    if b.GetIsAromatic():
        return 1.5
    t = b.GetBondType()
    if t == Chem.BondType.SINGLE:
        return 1.0
    if t == Chem.BondType.DOUBLE:
        return 2.0
    if t == Chem.BondType.TRIPLE:
        return 3.0
    return 0.0

def _ring_size_hist(m: Optional[Chem.Mol]) -> Tuple[Dict[int, int], int]:
    if m is None:
        return {5: 0, 6: 0, 7: 0, 8: 0}, 0
    ri = m.GetRingInfo()
    sizes = [len(r) for r in ri.AtomRings()]
    out = {5: 0, 6: 0, 7: 0, 8: 0}
    for s in sizes:
        if s in out:
            out[s] += 1
    return out, len(sizes)

def _ring_systems_count(m: Optional[Chem.Mol]) -> int:
    if m is None:
        return 0
    ri = m.GetRingInfo()
    rings = [set(r) for r in ri.AtomRings()]
    if not rings:
        return 0
    seen: set[int] = set()
    systems = 0
    for i in range(len(rings)):
        if i in seen:
            continue
        systems += 1
        stack = [i]
        seen.add(i)
        while stack:
            j = stack.pop()
            for k in range(len(rings)):
                if k in seen:
                    continue
                if rings[j] & rings[k]:
                    seen.add(k)
                    stack.append(k)
    return systems

def _murcko_stats(m: Optional[Chem.Mol]) -> Dict[str, int]:
    if m is None or MurckoScaffold is None:
        return {'MurckoAtoms': 0, 'MurckoRings': 0, 'MurckoRingSystems': 0, 'SideChainAtoms': 0 if m is None else m.GetNumAtoms()}
    try:
        scaf = MurckoScaffold.GetScaffoldForMol(m)
        if scaf is None or scaf.GetNumAtoms() == 0:
            return {'MurckoAtoms': 0, 'MurckoRings': 0, 'MurckoRingSystems': 0, 'SideChainAtoms': m.GetNumAtoms()}
        return {
            'MurckoAtoms': int(scaf.GetNumAtoms()),
            'MurckoRings': int(rdMolDescriptors.CalcNumRings(scaf)),
            'MurckoRingSystems': int(_ring_systems_count(scaf)),
            'SideChainAtoms': int(max(m.GetNumAtoms() - scaf.GetNumAtoms(), 0)),
        }
    except Exception:
        return {'MurckoAtoms': 0, 'MurckoRings': 0, 'MurckoRingSystems': 0, 'SideChainAtoms': m.GetNumAtoms()}

def augment_extra_cheaps(row: Dict[str, Any], m: Optional[Chem.Mol]) -> Dict[str, Any]:
    if m is None:
        row.update({'FracSingle': 0.0, 'FracDouble': 0.0, 'FracTriple': 0.0, 'FracAromatic': 0.0, 'MeanBondOrder': 0.0, 'UnsatBondCount': 0})
        row.update({'Rings5': 0, 'Rings6': 0, 'Rings7': 0, 'Rings8': 0, 'RingSystems': 0, 'Rings56_frac': 0.0})
        row.update({'FormalCharge': 0, 'IsZwitterion': 0})
        row.update(_estate_stats(m))
        row.update(_murcko_stats(m))
        row.update(_smiles_morphology(''))
        return row

    row.update(_estate_stats(m))
    bonds = list(m.GetBonds())
    nb = max(len(bonds), 1)
    n_single = sum(1 for b in bonds if b.GetBondType() == Chem.BondType.SINGLE and not b.GetIsAromatic())
    n_double = sum(1 for b in bonds if b.GetBondType() == Chem.BondType.DOUBLE)
    n_triple = sum(1 for b in bonds if b.GetBondType() == Chem.BondType.TRIPLE)
    n_arom = sum(1 for b in bonds if b.GetIsAromatic())
    row['FracSingle'] = n_single / nb
    row['FracDouble'] = n_double / nb
    row['FracTriple'] = n_triple / nb
    row['FracAromatic'] = n_arom / nb
    row['MeanBondOrder'] = (sum(_bond_order(b) for b in bonds) / nb) if nb else 0.0
    row['UnsatBondCount'] = int(n_double + n_triple + n_arom)

    hist, n_rings = _ring_size_hist(m)
    row['Rings5'] = int(hist[5])
    row['Rings6'] = int(hist[6])
    row['Rings7'] = int(hist[7])
    row['Rings8'] = int(hist[8])
    row['RingSystems'] = int(_ring_systems_count(m))
    row['Rings56_frac'] = (hist[5] + hist[6]) / (n_rings if n_rings > 0 else 1)

    row.update(_murcko_stats(m))
    tot_charge = sum(a.GetFormalCharge() for a in m.GetAtoms())
    has_pos = any(a.GetFormalCharge() > 0 for a in m.GetAtoms())
    has_neg = any(a.GetFormalCharge() < 0 for a in m.GetAtoms())
    row['FormalCharge'] = int(tot_charge)
    row['IsZwitterion'] = int(has_pos and has_neg)

    try:
        smi = Chem.MolToSmiles(m, canonical=True)
    except Exception:
        smi = ''
    row.update(_smiles_morphology(smi))
    return row

In [6]:
def _shape3d_from_cansmi(cansmi: str, maxIters: int = 0) -> Tuple[str, Dict[str, float]]:
    """Compute lean 3D shape features for ONE canonical SMILES."""
    try:
        m = Chem.MolFromSmiles(cansmi)
        if m is None:
            return cansmi, {}

        mH = Chem.AddHs(m)
        params = AllChem.ETKDGv3() if hasattr(AllChem, 'ETKDGv3') else AllChem.ETKDG()
        params.randomSeed = 123
        params.useRandomCoords = True

        with Chem.WrapLogs():
            cid = AllChem.EmbedMolecule(mH, params)
        if cid < 0:
            with Chem.WrapLogs():
                cid = AllChem.EmbedMolecule(mH, randomSeed=123)
            if cid < 0:
                return cansmi, {}

        if maxIters and maxIters > 0:
            try:
                with Chem.WrapLogs():
                    AllChem.UFFOptimizeMolecule(mH, confId=cid, maxIters=int(maxIters))
            except Exception:
                pass

        m_noH = Chem.RemoveHs(mH)
        confId = 0

        out: Dict[str, float] = {}
        for nm, fn in [
            ('RadiusOfGyration', rdMolDescriptors.CalcRadiusOfGyration),
            ('InertialShapeFactor', rdMolDescriptors.CalcInertialShapeFactor),
            ('PMI1', rdMolDescriptors.CalcPMI1),
            ('PMI2', rdMolDescriptors.CalcPMI2),
            ('PMI3', rdMolDescriptors.CalcPMI3),
            ('NPR1', rdMolDescriptors.CalcNPR1),
            ('NPR2', rdMolDescriptors.CalcNPR2),
        ]:
            try:
                with Chem.WrapLogs():
                    out[nm] = float(fn(m_noH, confId=confId))
            except Exception:
                out[nm] = 0.0

        pmi1 = out.get('PMI1', 0.0) or 0.0
        pmi2 = out.get('PMI2', 0.0) or 0.0
        pmi3 = out.get('PMI3', 0.0) or 0.0
        out['PMI2_over_PMI1'] = (pmi2 / pmi1) if pmi1 else 0.0
        out['PMI3_over_PMI1'] = (pmi3 / pmi1) if pmi1 else 0.0
        return cansmi, out
    except Exception:
        return cansmi, {}

def precompute_shape3d_cache(smiles_series: pd.Series, n_jobs: Optional[int] = None, maxIters: int = 0) -> Dict[str, Dict[str, float]]:
    n_jobs = n_jobs if n_jobs is not None else max(1, mp.cpu_count() - 1)
    can = smiles_series.astype(str).apply(lambda s: canonicalize_smiles(s))
    uniq = sorted(x for x in set(can.tolist()) if x is not None)
    if not uniq:
        return {}
    results = Parallel(n_jobs=n_jobs, backend='loky', batch_size=64)(
        delayed(_shape3d_from_cansmi)(s, maxIters=maxIters) for s in uniq
    )
    return {k: v for k, v in results if k is not None}

def rdkit_feature_row(m: Optional[Chem.Mol], compute_3d: bool = False, shape_cache: Optional[Dict[str, Dict[str, float]]] = None) -> Dict[str, Any]:
    row: Dict[str, Any] = {}

    for name, func in Descriptors._descList:
        row[name] = _safe(func, np.nan)(m)

    row['MolLogP'] = _safe(Crippen.MolLogP, np.nan)(m)
    row['MolMR'] = _safe(Crippen.MolMR, np.nan)(m)
    row['NumHAcceptors'] = _safe(Lipinski.NumHAcceptors, np.nan)(m)
    row['NumHDonors'] = _safe(Lipinski.NumHDonors, np.nan)(m)
    row['NumRings'] = _safe(rdMolDescriptors.CalcNumRings, 0)(m)
    row['NumAromaticRings'] = _safe(rdMolDescriptors.CalcNumAromaticRings, 0)(m)
    row['NumAliphaticRings'] = _safe(rdMolDescriptors.CalcNumAliphaticRings, 0)(m)
    row['NumSaturatedRings'] = _safe(rdMolDescriptors.CalcNumSaturatedRings, 0)(m)
    row['NumBridgeheadAtoms'] = _safe(rdMolDescriptors.CalcNumBridgeheadAtoms, 0)(m)
    row['NumSpiroAtoms'] = _safe(rdMolDescriptors.CalcNumSpiroAtoms, 0)(m)
    row['LargestRingSize'] = _safe(_largest_ring_size, 0)(m)
    row['NumAmideBonds'] = _safe(rdMolDescriptors.CalcNumAmideBonds, 0)(m)
    row['TPSA'] = _safe(rdMolDescriptors.CalcTPSA, np.nan)(m)
    row['LabuteASA'] = _safe(rdMolDescriptors.CalcLabuteASA, np.nan)(m)

    if m is None:
        for el in ['C','H','N','O','S','F','Cl','Br','I','P']:
            row[f'Count_{el}'] = 0
    else:
        for el in ['C','N','O','S','F','Cl','Br','I','P']:
            row[f'Count_{el}'] = _count_atoms(m, [el])
        row['Count_H'] = _safe(count_explicit_h, 0)(m)

    for attr in dir(Fragments):
        if attr.startswith('fr_'):
            fn = getattr(Fragments, attr)
            if callable(fn):
                row[attr] = _safe(fn, 0)(m)

    try:
        if m is not None:
            mgen = rdFingerprintGenerator.GetMorganGenerator(radius=MORGAN_RADIUS, fpSize=MORGAN_BITS, countSimulation=False)
            mfp = mgen.GetFingerprint(m)
            for i in range(MORGAN_BITS):
                row[f'Morgan_{i}'] = int(mfp[i])
        else:
            for i in range(MORGAN_BITS):
                row[f'Morgan_{i}'] = 0
        if USE_MACCS:
            if m is not None:
                maccs = GenMACCSKeys(m)
                for i in range(len(maccs)):
                    row[f'MACCS_{i}'] = int(maccs[i])
            else:
                for i in range(167):
                    row[f'MACCS_{i}'] = 0
    except Exception:
        for i in range(MORGAN_BITS):
            row.setdefault(f'Morgan_{i}', 0)
        if USE_MACCS:
            for i in range(167):
                row.setdefault(f'MACCS_{i}', 0)

    for vsa_name, vsa_fn in [
        ('SlogP_VSA', getattr(rdMolDescriptors, 'SlogP_VSA_', None)),
        ('SMR_VSA', getattr(rdMolDescriptors, 'SMR_VSA_', None)),
        ('EState_VSA', getattr(rdMolDescriptors, 'EState_VSA_', None)),
    ]:
        if vsa_fn is None or m is None:
            continue
        try:
            bins = vsa_fn(m)
            for i, val in enumerate(bins):
                row[f'{vsa_name}{i}'] = float(val)
            row[f'{vsa_name}_sum'] = float(sum(bins))
        except Exception:
            pass

    row.update(_safe(gasteiger_stats, {})(m))

    if compute_3d and m is not None:
        try:
            cansmi = Chem.MolToSmiles(m, canonical=True)
        except Exception:
            cansmi = None
        if shape_cache is not None and cansmi in shape_cache:
            row.update(shape_cache[cansmi])
        elif cansmi is not None:
            _, quick = _shape3d_from_cansmi(cansmi, maxIters=MAX_ITERS_3D)
            row.update(quick)

    hbd = float(row.get('NumHDonors', 0.0) or 0.0)
    hba = float(row.get('NumHAcceptors', 0.0) or 0.0)
    mw = float(row.get('MolWt', 0.0) or 0.0)
    hat = float(row.get('HeavyAtomCount', 0.0) or 1.0)
    tpsa = float(row.get('TPSA', 0.0) or 0.0)
    nrot = float(row.get('NumRotatableBonds', 0.0) or 0.0)
    narm = float(row.get('NumAromaticRings', 0.0) or 0.0)
    mollogp = float(row.get('MolLogP', 0.0) or 0.0)
    bertz = float(row.get('BertzCT', 0.0) or 0.0)

    row['HBondCapacity'] = hbd + hba
    row['HBondDensity_perHeavyAtom'] = (hbd + hba) / hat
    row['RingDensity_perHeavyAtom'] = float(row.get('NumRings', 0.0) or 0.0) / hat
    row['HalogenCount'] = float(row.get('Count_F', 0) + row.get('Count_Cl', 0) + row.get('Count_Br', 0) + row.get('Count_I', 0))
    row['HeteroAtomFrac'] = float(row.get('Count_N', 0) + row.get('Count_O', 0) + row.get('Count_S', 0) + row.get('Count_P', 0)) / hat
    row['AromRingFrac'] = float(row.get('NumAromaticRings', 0.0) or 0.0) / float((row.get('NumRings', 1.0) or 1.0))

    row['HBond_Product'] = hbd * hba
    row['LogP_div_TPSA'] = mollogp / (tpsa + 1.0)
    row['LogP_x_TPSA'] = mollogp * tpsa
    row['Flexibility_Score'] = nrot / (mw + 1.0)
    row['MolWt_x_AromaticRings'] = mw * narm
    row['Complexity_per_MW'] = bertz / (mw + 1.0)
    row['Rigidity_Score'] = narm / (nrot + 1.0)

    row = augment_extra_cheaps(row, m)
    return row

def featurize_smiles(df: pd.DataFrame, compute_3d: bool = True, maxIters_3d: int = 0) -> pd.DataFrame:
    shape_cache = None
    if compute_3d:
        shape_cache = precompute_shape3d_cache(df['SMILES'], n_jobs=-1, maxIters=maxIters_3d)
    mols = df['SMILES'].apply(lambda s: Chem.MolFromSmiles(s) if pd.notna(s) else None)
    feats = [rdkit_feature_row(m, compute_3d=compute_3d, shape_cache=shape_cache) for m in mols]
    feats = pd.DataFrame(feats)
    feats = drop_constant_and_duplicate_columns(feats)
    out = pd.concat([df.reset_index(drop=True), feats.reset_index(drop=True)], axis=1)
    out = reduce_memory_usage(out)
    return out

In [7]:
df_train_feat = featurize_smiles(df_train, compute_3d=COMPUTE_3D, maxIters_3d=MAX_ITERS_3D)
print('train featurized:', df_train_feat.shape)

df_test_feat = None
if df_test is not None:
    df_test_feat = featurize_smiles(df_test, compute_3d=COMPUTE_3D, maxIters_3d=MAX_ITERS_3D)
    print('test featurized:', df_test_feat.shape)
    extra_train = set(df_train_feat.columns) - set(df_test_feat.columns) - {'Tm'}
    extra_test = set(df_test_feat.columns) - set(df_train_feat.columns)
    print('extra train-only cols:', len(extra_train), 'extra test-only cols:', len(extra_test))
    df_train_feat = df_train_feat.drop(columns=list(extra_train))
    df_test_feat = df_test_feat.drop(columns=list(extra_test))
    print('aligned:', df_train_feat.shape, df_test_feat.shape)

numeric_cols = list(set(df_train_feat.select_dtypes(include=['number']).columns) - {'Tm'})
df_train_feat[numeric_cols] = df_train_feat[numeric_cols].fillna(df_train_feat[numeric_cols].mean()).fillna(0)
if df_test_feat is not None:
    df_test_feat[numeric_cols] = df_test_feat[numeric_cols].fillna(df_train_feat[numeric_cols].mean()).fillna(0)

display(df_train_feat.head(2))

train featurized: (2660, 1329)


,id,SMILES,Tm,Group 1,Group 2,Group 3,Group 4,Group 5,Group 6,Group 7,...,Rings6,Rings7,Rings8,RingSystems,Rings56_frac,MurckoAtoms,SideChainAtoms,SMI_len,SMI_branches,SMI_ringDigits
0,2175,FC1=C(F)C(F)(F)C1(F)F,213.149994,0,0,0,0,0,0,0,...,0,0,0,1,0.0,4,6,21,4,2
1,1222,c1ccc2c(c1)ccc1[nH]c3ccccc3c12,407.149994,0,0,0,0,0,0,0,...,3,0,0,1,1.0,17,0,30,1,8


In [8]:
csv_path = OUTPUT_DIR / 'melting_point_features.csv'
parquet_path = OUTPUT_DIR / 'melting_point_features.parquet'

df_train_feat.to_csv(csv_path, index=False)
print(f'Saved train features to {csv_path}')

try:
    df_train_feat.to_parquet(parquet_path, index=False)
    print(f'Saved train features to {parquet_path}')
except Exception as exc:
    print(f'Skipped parquet export: {exc}')

if df_test_feat is not None:
    test_csv_path = OUTPUT_DIR / 'melting_point_features_test.csv'
    df_test_feat.to_csv(test_csv_path, index=False)
    print(f'Saved test features to {test_csv_path}')

Saved train features to result/data/melting_point_features.csv
Skipped parquet export: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.


# Baseline model (LightGBM)
This section trains a simple baseline on the generated features to get first-pass metrics (MAE/RMSE/R²).
- Uses a train/valid split.
- Fits imputation on train only (prevents leakage).
- Saves metrics + predictions to `result/data/`.

In [9]:
# Install baseline modeling deps if missing (best-effort)
for pkg in [
    ('scikit-learn', 'scikit-learn'),
    ('lightgbm', 'lightgbm'),
]:
    try:
        ensure_package(pkg[0], pkg[1])
    except Exception as exc:
        print(f"Package install check failed for {pkg}: {exc}")

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

scikit-learn already installed.
lightgbm already installed.
lightgbm already installed.


In [10]:
# Build X/y from the featurized training frame
assert 'Tm' in df_train_feat.columns, "Expected target column 'Tm' in df_train_feat"
assert 'SMILES' in df_train_feat.columns, "Expected 'SMILES' column in df_train_feat"

work_df = df_train_feat.copy()
work_df = work_df.dropna(subset=['Tm']).reset_index(drop=True)

# Use numeric features only (exclude identifiers/strings)
feature_cols = [c for c in work_df.columns if c not in ('Tm', 'SMILES')]
X = work_df[feature_cols].select_dtypes(include=['number'])
y = work_df['Tm'].astype(float)

print('Rows used:', len(work_df))
print('Numeric features:', X.shape[1])
print('Target stats:', float(y.min()), float(y.mean()), float(y.max()))

Rows used: 2660
Numeric features: 1327
Target stats: 53.540000915527344 278.3451694631935 897.1500244140625


In [11]:
 # Outlier removal on target (IQR rule) + report percent removed
tm = work_df['Tm'].astype(float)
q1 = float(tm.quantile(0.25))
q3 = float(tm.quantile(0.75))
iqr = q3 - q1
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

mask = (tm >= lower) & (tm <= upper)
removed = int((~mask).sum())
total = int(len(work_df))
pct_removed = 100.0 * removed / total if total else 0.0

print(f"IQR bounds for Tm: [{lower:.3f}, {upper:.3f}] (Q1={q1:.3f}, Q3={q3:.3f}, IQR={iqr:.3f})")
print(f"Removed outliers: {removed}/{total} ({pct_removed:.2f}%)")

# Apply filter
work_df = work_df.loc[mask].reset_index(drop=True)
feature_cols = [c for c in work_df.columns if c not in ('Tm', 'SMILES')]
X = work_df[feature_cols].select_dtypes(include=['number'])
y = work_df['Tm'].astype(float)

print('After outlier removal -> Rows:', len(work_df), 'Numeric features:', X.shape[1])

IQR bounds for Tm: [55.037, 487.338] (Q1=217.150, Q3=325.225, IQR=108.075)
Removed outliers: 48/2660 (1.80%)
After outlier removal -> Rows: 2612 Numeric features: 1327


In [12]:
# Train/valid split + train-only imputation (no leakage)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
 )

imputer = SimpleImputer(strategy='median')
X_train_imp = imputer.fit_transform(X_train)
X_valid_imp = imputer.transform(X_valid)

model = lgb.LGBMRegressor(
    objective='regression',
    n_estimators=2000,
    learning_rate=0.03,
    num_leaves=63,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
)

model.fit(
    X_train_imp, y_train,
    eval_set=[(X_valid_imp, y_valid)],
    eval_metric='l1',
    callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)],
)

pred_valid = model.predict(X_valid_imp, num_iteration=model.best_iteration_)
mae = float(mean_absolute_error(y_valid, pred_valid))
mse = float(mean_squared_error(y_valid, pred_valid))
rmse = float(np.sqrt(mse))
r2 = float(r2_score(y_valid, pred_valid))

print('Baseline metrics')
print('MAE :', mae)
print('RMSE:', rmse)
print('R2  :', r2)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.830627 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 23009
[LightGBM] [Info] Number of data points in the train set: 2089, number of used features: 739
[LightGBM] [Info] Start training from score 273.667033
Baseline metrics
MAE : 24.851536147016613
RMSE: 34.35449452128298
R2  : 0.7975558272206789
Baseline metrics
MAE : 24.851536147016613
RMSE: 34.35449452128298
R2  : 0.7975558272206789


/home/tp_ubuntu/project/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [13]:
# Save baseline outputs
baseline_metrics = {
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'n_rows': int(len(work_df)),
    'n_features': int(X.shape[1]),
    'best_iteration': int(getattr(model, 'best_iteration_', 0) or 0),
}

metrics_path = OUTPUT_DIR / 'baseline_lgbm_metrics.json'
with open(metrics_path, 'w', encoding='utf-8') as f:
    json.dump(baseline_metrics, f, indent=2)
print('Saved metrics to', metrics_path)

pred_path = OUTPUT_DIR / 'baseline_lgbm_predictions.csv'
pred_df = pd.DataFrame({
    'y_true': y_valid.reset_index(drop=True),
    'y_pred': pred_valid,
})
pred_df.to_csv(pred_path, index=False)
print('Saved predictions to', pred_path)

Saved metrics to result/data/baseline_lgbm_metrics.json
Saved predictions to result/data/baseline_lgbm_predictions.csv
